# Scrapy + Scraper API

In [1]:
pip install scrapy

     |████████████████████████████████| 239 kB 17.5 MB/s eta 0:00:01
     |████████████████████████████████| 236 kB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 47.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 30.3 MB/s eta 0:00:01��    | 4.8 MB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 38.0 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 5.3 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 698 kB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 4.3 MB/s  eta 0:00:01
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11515 sha256=60c75b175c0caf2fa5378f0cf51aaf48a4a2e57928965ebcb192a3fd788f7934
  Stored in directory: /home/jovyan/.cache/pip/wheels/28/db/61/691c759da06ba9b86da079bdd17cb3e01828d49d5c152cb3af
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl 

In [2]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = '6e2f13665a14d50e37acdfc9636b4877'

In [3]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'http://www.google.com/search?' + urlencode(google_dict)
   return 'http://www.google.com/search?' + urlencode(google_dict)

class GoogleSpider(scrapy.Spider):
   name = 'google'
   #allowed_domains = ['api.scraperapi.com']
   #custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
   allowed_domains = ['www.google.com']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}

   def start_requests(self):
       queries = ['innovation', 'gestion'] 
       for query in queries:
           url = create_google_url(query)
           yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0})

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [4]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests())

In [5]:
result

[<GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr>,
 <GET http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dgestion%26num%3D100&autoparse=true&country_code=fr>]

In [6]:
link = []
for i in range(len(result)):
    link = link + [str(result[i])[5:-1]]
print(link)

['http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr', 'http://api.scraperapi.com/?api_key=6e2f13665a14d50e37acdfc9636b4877&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dgestion%26num%3D100&autoparse=true&country_code=fr']


In [7]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
for i in range(len(url)):
    response = get(url[i])
source = None # Le code source de la page 
print(response.status_code)
if response.status_code == 200 :
    # Si la requete s'est bien passee
    source = response.text

200


In [8]:
source

'{"search_information":{"total_results":694000000,"time_taken_displayed":1.1,"query_displayed":"gestion"},"ads":[],"knowledge_graph":{"title":"","description":"","source":{"name":""},"related":[],"related_link":null,"social_media":[],"see_more_about":[]},"related_questions":[],"answer_box":null,"organic_results":[{"position":1,"title":"Définitions : gestion - Dictionnaire de français Larousse","snippet":"gestion - Définitions Français : Retrouvez la définition de gestion... - synonymes, homonymes, difficultés, citations.","link":"https://www.larousse.fr/dictionnaires/francais/gestion/36853","date":"","displayed_link":"www.larousse.fr › francais › gestion","thumbnail":null,"sitelinks":{"inline":[],"block":[]},"cached_page_link":"https://webcache.googleusercontent.com/search?q=cache:UxliNtC1Y_QJ:https://www.larousse.fr/dictionnaires/francais/gestion/36853+&cd=1&hl=en&ct=clnk&gl=fr"},{"position":2,"title":"Gestion : Définition simple et facile du dictionnaire - L\'Internaute","snippet":"G

In [9]:
import re
s = source
pattern = re.compile("\"link\"[^,]+")
my_links = list(set([x for x in pattern.findall(s)]))
my_links = [i[8:] for i in my_links]
my_links = [i[:-1] for i in my_links]
mylinks=[]

for i in my_links:
    if "https://www.google" not in i:
        mylinks.append(i)

mylinks

['https://www.a-gestion.fr/',
 'https://www.solutionsimmobilierbysoprasteria.com/plateforme/',
 'https://www.wordreference.com/fren/gestion',
 'https://www.cdg69.fr/',
 'https://www.magestionlocative.fr/',
 'https://www.merriam-webster.com/dictionary/gestion',
 'https://www.interieur.gouv.fr/Le-ministere/Securite-civile',
 'https://www.gererseul.com/',
 'https://www.orleans-metropole.fr/dechets',
 'http://fncdg.com/',
 'https://progestion.fr/',
 'https://diplomeo.com/formations-metier-gestion',
 'http://gestion-21.fr/',
 'https://www.cncgp.fr/',
 'http://www.cdg33.fr/',
 'https://www.gestiondefortune.com/',
 'https://www.revuegestion.ca/',
 'https://www.oui.sncf/informations-legales/confidentialite/gestion-cookies',
 'https://www.finance-gestion.com/',
 'https://www.france-gestion.fr/',
 'https://www.cartebtp.fr/connexion.html',
 'https://www.vivinter.fr/fr/accueil-vivinter-centre-de-gestion.html',
 'https://www.linternaute.fr/dictionnaire/fr/definition/gestion/',
 'https://www.atrium-